In [202]:
import pandas as pd
import numpy as np
dataset_sin = pd.read_csv("export/dataset_without_resume.csv", delimiter=',', on_bad_lines='skip')
# imdb_resumes_bis.json
imdb_resumes = pd.read_json("export/imdb_resumes_bis.json")


In [203]:
import re

pd.options.display.max_rows = None
pd.options.display.max_seq_items = None

for i in range(len(imdb_resumes)-1): # 
    raw = imdb_resumes.iloc[i]

    values = [ raw['duree'], raw['genre'], raw['parental_advisor'] ]
    
    imdb_resumes.genre[i] = None
    imdb_resumes.duree[i] = None
    imdb_resumes.parental_advisor[i] = None

    for item in values:

        if type(item) is str :
            if '\n' in item:
                imdb_resumes.genre[i] = item
            elif 'min' in item:
                imdb_resumes.duree[i] = item
            else :
                imdb_resumes.parental_advisor[i] = item

In [204]:
imdb_resumes['duree'] = imdb_resumes['duree'].replace(r'\D+', '', regex=True)

imdb_resumes['genre'] = imdb_resumes['genre'].str.replace(r'\n', '')

imdb_resumes['resume'] = imdb_resumes['resume'].str.replace(r'\n', '')

imdb_resumes['duree'] = [ '0' if x == None else x for x in imdb_resumes['duree']]
imdb_resumes['duree'] = imdb_resumes['duree'].astype(int)
imdb_resumes['duree'] = [ np.nan if x == 0 else x for x in imdb_resumes['duree']]
imdb_resumes = imdb_resumes.dropna(subset=['duree'])

imdb_resumes = imdb_resumes.drop_duplicates(subset='imdb_search', keep="first")
imdb_resumes = imdb_resumes.reset_index().drop(columns=["index"])

<ipython-input-204-a3388da07ecf>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  imdb_resumes['genre'] = imdb_resumes['genre'].str.replace(r'\n', '')
<ipython-input-204-a3388da07ecf>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  imdb_resumes['resume'] = imdb_resumes['resume'].str.replace(r'\n', '')


In [205]:
dataset = pd.merge(dataset_sin, imdb_resumes, left_on='imdb_search', right_on='imdb_search')
dataset = dataset.drop(columns=['imdb_search','genre', 'numVotes'])
for i in range( len(dataset)-1 ):
    dataset['resume'] = [elem.replace('  ', '') for elem in dataset['resume']]
for i in range( len(dataset)-1 ):
    if dataset.iloc[i]['resume'] == ' ':
        print(i)
        dataset = dataset.drop([i])
    elif dataset.iloc[i]['resume'] == '':
        print(i)
        dataset = dataset.drop([i])
dataset.head()

,originalTitle,startYear,genres,averageRating,primaryName,parental_advisor,duree,resume
0,Miss Jerry,1894,Romance,5.3,Alexander Black,None,45.0,The adventures of a female reporter in the 1890s.
1,The Story of the Kelly Gang,1906,"Action,Adventure,Biography",6.0,Charles Tait,Not Rated,70.0,"Story of Ned Kelly, an infamous 19th-century A..."
2,L'enfant prodigue,1907,Drama,4.7,Michel Carré,None,90.0,It's a play in three parts. This film is suppo...
3,Ma l'amor mio non muore...,1914,Drama,6.2,Mario Caserini,None,90.0,"Leslie Swayne, an adventurer, in order to obta..."
4,Gli ultimi giorni di Pompei,1913,"Adventure,Drama",6.1,Mario Caserini,None,88.0,Two love triangles intersect in ancient Pompei.


In [206]:
# Convert string Genre to list
dataset["genres"] = [ str(x).replace(' ', '') for x in dataset["genres"][:] ] 
dataset["genres"] = [ x.split(",") for x in dataset["genres"][:] ]

# https://www.sbert.net/docs/quickstart.html#comparing-sentence-similarities
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')
# from unittest import result

dataset['genre'] = ''
dataset['themaScore'] = 0.0

for i in range( len(dataset) ):
    resume = dataset.iloc[i]['resume']
    genres = dataset.iloc[i]['genres']
    bestScore = 0.0
    bestGenre = ''
    for genre in genres:
        emb1 = model.encode(str(genre))
        emb2 = model.encode(str(resume))
        cos_sim = util.cos_sim(emb1, emb2)
        result = cos_sim.tolist()[0][0]
        if result > bestScore:
            bestScore = result
            bestGenre = genre
    # print(i, '/', len(dataset))
    dataset.themaScore[i] = bestScore
    dataset.genre[i] = bestGenre
dataset = dataset.drop(columns=['genres'])

<ipython-input-206-1222c0ca889a>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.themaScore[i] = bestScore
<ipython-input-206-1222c0ca889a>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.genre[i] = bestGenre


In [207]:
dataset = dataset.rename(columns={"originalTitle": "title", "startYear": "year", "primaryName" : "director", "parental_advisor" : "parentalAdvisor", "averageRating" : "imdbRating"})

In [208]:
dataset.to_csv(r'export/dataset.csv', index = False, header=True)